In [ ]:
import tensorflow as tf
import cv2
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime
from preprocessing import combine_data_frompath
import datetime
from tensorflow.keras import callbacks



In [ ]:
train_data = combine_data_frompath('aug_data\\train\\labels\\*.json','aug_data\\train\\images\\*.jpg')
val_data = combine_data_frompath('aug_data\\val\\labels\\*.json','aug_data\\val\\images\\*.jpg')

In [ ]:
from tensorflow.keras.applications import VGG16
vgg16 = VGG16(include_top=False)

In [ ]:
X = train_data.take(1).as_numpy_iterator()

In [ ]:
X = X.next()

In [ ]:
Y = X[0][0]

In [ ]:
Y.shape

In [ ]:
D =vgg16(Y[np.newaxis])

In [ ]:
D.shape

In [ ]:
from tensorflow.keras.layers import Conv2D

In [ ]:
layer = Conv2D(2048,kernel_size=(3,3),padding="same")

In [ ]:
layer(D)

In [ ]:
layer2 = Conv2D(1,kernel_size=(3,3))

In [ ]:
layer2(layer(D))

In [ ]:
from model import ConvFaceTracker
facetracker = ConvFaceTracker()

In [ ]:
Y.shape

In [ ]:
Y1 = facetracker(Y[np.newaxis])

In [ ]:
Y1

In [ ]:
Y2

In [ ]:
from model import FaceTracker
facetracker2 = FaceTracker()

In [ ]:
Y2 = facetracker2(Y[np.newaxis])

In [ ]:
Y1

In [ ]:
Y2

In [ ]:
Y1[0][:,0,0]

In [ ]:
tf.squeeze(tf.squeeze(Y1[0],axis=-1),axis=-1)

In [ ]:
import numpy as np

In [ ]:
a = np.arange(16).reshape(1,4,4,1)
a

In [ ]:
a.shape[0]

In [ ]:
a[:,1,1]

In [1]:
import tensorflow as tf
import cv2
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime
from preprocessing import combine_data_frompath
import datetime
from tensorflow.keras import callbacks



In [2]:
train_data = combine_data_frompath('aug_data\\train\\labels\\*.json','aug_data\\train\\images\\*.jpg')
val_data = combine_data_frompath('aug_data\\val\\labels\\*.json','aug_data\\val\\images\\*.jpg')

In [3]:
from model import FaceTracker
facetracker = FaceTracker()

In [4]:
batches_per_epoch = len(train_data)
lr_decay = (1./0.75 -1)/batches_per_epoch
opt = tf.keras.optimizers.Adam(learning_rate=0.0001, decay=lr_decay)
facetracker.compile(optimizer=opt)

In [5]:
early_stopping_callback = callbacks.EarlyStopping(monitor="val_total_loss",patience=3)
num_epochs = 20

In [6]:
facetracker.fit(train_data,epochs=num_epochs,
                callbacks=[early_stopping_callback],
                validation_data=val_data,
                verbose=1)

Epoch 1/20
37/37 [==============================] - 26s 520ms/step - total_loss: 3.5227 - classloss: 0.3247 - coordloss: 3.1981 - val_total_loss: 0.1273 - val_classloss: 0.0039 - val_coordloss: 0.1234
Epoch 2/20
37/37 [==============================] - 16s 438ms/step - total_loss: 0.5638 - classloss: 0.0140 - coordloss: 0.5498 - val_total_loss: 0.1390 - val_classloss: 7.3188e-04 - val_coordloss: 0.1383
Epoch 3/20
37/37 [==============================] - 16s 437ms/step - total_loss: 0.1497 - classloss: 4.3886e-04 - coordloss: 0.1493 - val_total_loss: 0.0335 - val_classloss: 3.9595e-05 - val_coordloss: 0.0335
Epoch 4/20
37/37 [==============================] - 16s 437ms/step - total_loss: 0.0615 - classloss: 8.8337e-05 - coordloss: 0.0614 - val_total_loss: 0.0121 - val_classloss: 4.5526e-05 - val_coordloss: 0.0121
Epoch 5/20
37/37 [==============================] - 16s 436ms/step - total_loss: 0.0383 - classloss: 5.4196e-05 - coordloss: 0.0383 - val_total_loss: 0.0067 - val_classloss: 3.

In [7]:
cap = cv2.VideoCapture(0)
while cap.isOpened():
    _ , frame = cap.read()
    frame = frame[50:500, 50:500,:]
    
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    resized = tf.image.resize(rgb, (120,120))
    
    yhat = facetracker.predict(np.expand_dims(resized/255,0))
    sample_coords = yhat[1][0]
    
    if yhat[0] > 0.5: 
        # Controls the main rectangle
        cv2.rectangle(frame, 
                      tuple(np.multiply(sample_coords[:2], [450,450]).astype(int)),
                      tuple(np.multiply(sample_coords[2:], [450,450]).astype(int)), 
                            (255,0,0), 2)
        # Controls the label rectangle
        cv2.rectangle(frame, 
                      tuple(np.add(np.multiply(sample_coords[:2], [450,450]).astype(int), 
                                    [0,-30])),
                      tuple(np.add(np.multiply(sample_coords[:2], [450,450]).astype(int),
                                    [80,0])), 
                            (255,0,0), -1)
        
        # Controls the text rendered
        cv2.putText(frame, 'face', tuple(np.add(np.multiply(sample_coords[:2], [450,450]).astype(int),
                                               [0,-5])),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
    
    cv2.imshow('EyeTrack', frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 19ms/step
